In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import os
import re

In [43]:
def Binary_orbit(time,asini,porb,ecc,omega_d,t0=-1,t90=-1,pporb=0.0,limit=1.0*10**-10,maxiter=40):

	time=np.array(time)  #time should be an array to prevent round-off errors
	if t90==-1 and t0==-1:
		print("error: need t90 or t0 value")
		return

	if t90!=-1 and t0!=-1:
		print("error: Only one of the t90 and t0 arguments is allowed")
		return

	if ecc<0 :
		print("error: eccentricity must be positive!")
		return
	if ecc>=1:
		print("error: Orbit correction has only been implemented for circular and elliptic orbits")
		return
	if ecc==0:
		omega_d=0 #circular orbit
	if t0 ==-1:
		t0 = t90+(omega_d-90.)/360. * porb
		print(t0,"Debugging time")
		
	if maxiter <=0:
		maxiter=20

	asini_d = asini/86400. #86400 seconds in a day, unit is light day
	t= time

	#Corrections for eccentricity 0<=ecc<1
	omega = omega_d * np.pi/180.0 # Radians conversion
	sinw = np.sin(omega)
	cosw = np.cos(omega)
	sq = ((1.-ecc)*(1.+ecc))**0.5
	cor =np.array([2.*limit]*len(t))
	

	#start with number of iterations = zero
	numiter=0
			
	contada=0
	print('starting iterations (max 20)')
	while((abs(np.amax(cor)) > limit) and (numiter < maxiter)):
		tper = (t-t0)/porb
		#print(tper)
		# Calculate the mean anomaly
		m = 2 * np.pi * (tper * (1. - 0.5 * pporb * tper))
		#print(m)
		# Normalize m to the range [0, 2π]
		m = m % (2 * np.pi)

		# Shift m to the range [-π, π] if needed
		m[m > np.pi] -= 2 * np.pi
		#print(m)
		m=np.array(m)

		print("Mean Anomaly:" ,m)
		
		eanom=np.array([1.0]*len(t))
		eanom = KeplerEquation1(m,ecc)  #use this command for a better solution
		
		print("Eccentric Anomaly:", eanom)  # Debugging line

		sin_e = np.sin(eanom)
		cos_e = np.cos(eanom)
		print("e: ",ecc,"a_sin_i",asini_d,"omega ",omega)
		print("sinE: ",sin_e)
		print("cosE: ",cos_e)
		z = asini_d*(sinw*(cos_e-ecc)+sq*cosw*sin_e) ##Romer Delay Term
		#print("correction factor(z): ",z*86400)
		f = (t-time)+z  
		print("term f: ",f)                             
		df = (sq*cosw*cos_e - sinw*sin_e)*(2*np.pi*asini_d/(porb*(1.0-ecc*cos_e)))
		print("term df:",df)
		cor =f/(1.0+df)
		print("Final Correction: ",cor)
		t = t-cor #### Altering it to add this time 
		numiter=numiter+1
		contada=contada+1
		print('iteration ' + str(contada)+' completed (max 20)')
		if numiter >= maxiter:
			print("Exceeded maxiter iterations and did not reach convergence");
			break
	return(t)


In [ ]:
def KeplerEquation1(m,ecc):
	m=np.array(m)
	if ecc<0 :
		print("error: eccentricity must be positive!")
		return
	if ecc>=1:
		print("error: Orbit correction has only been implemented for circular and elliptic orbits")
	
	#print("Mean anomaly input to kepler:", m)
	for j in range(0,len(m)):
		mod_m=m[j]/2/np.pi
		m[j]=m[j]-2*np.pi*round(mod_m)
		
		while m[j]>np.pi:
			m[j]=m[j]-2*np.pi
		while m[j]<-np.pi:
			m[j]=m[j]+2*np.pi
	if ecc==0:
		E=m
		
	aux=4.0*ecc+0.5
	alpha=(1.0-ecc)/aux
	
	Beta=m/(2.0*aux)
	aux=np.sqrt(Beta**2+alpha**3)
	
	z=Beta+aux
	test=np.array([1.0]*len(z))
	for j in range(0,len(m)):
		if z[j]<=0.0:
			z[j]=Beta[j]-aux[j]
			
		test[j]=abs(z[j])**(1/3)
	z=test
	for j in range(0,len(m)):
		if z[j]<0.0:
			z[j]=-z[j]
	s0=z-alpha/z
	
	s1=s0-(0.078*s0**5)/(1.0+ecc)
	e0=m+ecc*(3.0*s1-4.0*s1**3)	
	
	se0=np.sin(e0)
	ce0=np.cos(e0)
	
	##Newton Raphson method
	f  = e0-ecc*se0-m
	f1 = 1.0-ecc*ce0
	f2 = ecc*se0
	f3 = ecc*ce0
	f4 = -f2
	u1 = -f/f1
	u2 = -f/(f1+0.50*f2*u1)
	u3 = -f/(f1+0.50*f2*u2+.16666666666667*f3*u2*u2)
	u4 = -f/(f1+0.50*f2*u3+.16666666666667*f3*u3*u3+.041666666666667*f4*u3**3)
	
	eccanom=e0+u4
	
	for j in range(0,len(m)):
		while eccanom[j]>=2.0*np.pi:
			eccanom[j]=eccanom[j]-2.0*np.pi
		while eccanom[j]<2.0*np.pi:
			eccanom[j]=eccanom[j]+2.0*np.pi
	##better solution
	#print("ecc anomaly before iteration inside kepler equation :", eccanom)

	CONT=True
	thresh=10**-3
	for j in range(0,len(m)):
		if m[j]<0:
			m[j]=m[j]+2.0*np.pi
	diff=eccanom-np.sin(eccanom)-m
	for j in range(0,len(m)):
		if abs(diff[j])>10**-10:
			I=diff[j]
			while CONT==True:
				fe=eccanom[j]-ecc*np.sin(eccanom[j])-m[j]
				fs=1.0-ecc*np.cos(eccanom[j])
				oldval=eccanom[j]
				eccanom[j]=oldval-fe/fs
				if abs(oldval-eccanom[j])<thresh :CONT=False
			while eccanom[j]>= np.pi:eccanom[j]=eccanom[j]-2.0*np.pi
			while eccanom[j]< np.pi:eccanom[j]=eccanom[j]+2.0*np.pi

	# Iterate until convergence for each m value
	thresh = 1e-6  # Convergence threshold
	for j in range(len(m)):
        # Iterative correction (Newton-Raphson method)
		max_iter = 40  # Maximum number of iterations for each eccentric anomaly
		iter_count = 0
		
		while iter_count < max_iter:
            # Calculate the difference (Kepler's equation)
			fe = eccanom[j] - ecc * np.sin(eccanom[j]) - m[j]
			fs = 1.0 - ecc * np.cos(eccanom[j])

			# If fs is too small, break out to avoid division by zero
			if abs(fs) < 1e-10:
				break

			# Update the Eccentric Anomaly using Newton-Raphson
			new_eccanom = eccanom[j] - fe / fs

			# Check for convergence
			if abs(new_eccanom - eccanom[j]) < thresh:
				eccanom[j] = new_eccanom
				break
			eccanom[j] = new_eccanom
			iter_count += 1

        # If maximum iterations are exceeded, print a warning
		if iter_count >= max_iter:
			print(f"Warning: Did not converge for m[{j}] after {max_iter} iterations.")

		# Ensure the final value is within [-pi, pi]
		while eccanom[j] >= np.pi:
			eccanom[j] -= 2 * np.pi
		while eccanom[j] < -np.pi:
			eccanom[j] += 2 * np.pi
	return eccanom


In [45]:
def find_event_files(base_directory="."):
    event_files = []
    
    # Regular expression for ProposalID (P followed by digits)
    proposal_pattern = re.compile(r"^P\d+$")
    
    # Regular expression for ObservationID (P followed by digits, likely matching ProposalID prefix)
    #observation_pattern = re.compile(r"^P\d+$")
    observation_pattern = re.compile(r"^P0504196019")

    # Regular expression for ExposureID (P followed by digits, extended further)
    exposure_pattern = re.compile(r"^P\d+$")
    
    # Regular expression for instrument-specific FITS files
    file_pattern = re.compile(r"^P\d+_HE_screen\.fits$|^P\d+_ME_screen\.fits$|^P\d+_LE_screen\.fits$")
    
    # Traverse base directory
    for proposal_dir in os.listdir(base_directory):
        proposal_path = os.path.join(base_directory, proposal_dir)
        
        
        if os.path.isdir(proposal_path) and proposal_pattern.match(proposal_dir):
            for observation_dir in os.listdir(proposal_path):
                observation_path = os.path.join(proposal_path, observation_dir)
                
                
                if os.path.isdir(observation_path) and observation_pattern.match(observation_dir):
                    pipeline_path = os.path.join(observation_path, "pipeline_output")

                    if os.path.isdir(pipeline_path):
                        for file in os.listdir(pipeline_path):
                            if file_pattern.match(file):
                                full_path = os.path.join(pipeline_path, file)
                                event_files.append(full_path)
    
    return event_files

#Example usage:
gti_files = find_event_files(".")
print(gti_files)


['./P0504196/P0504196019/pipeline_output/P050419601901_ME_screen.fits', './P0504196/P0504196019/pipeline_output/P050419601901_LE_screen.fits', './P0504196/P0504196019/pipeline_output/P050419601901_HE_screen.fits']


In [46]:
def find_gti_files(base_directory="."):
    gti_files = []
    
    # Regular expression for ProposalID (P followed by digits)
    proposal_pattern = re.compile(r"^P\d+$")
    
    # Regular expression for ObservationID (Modify as needed)
    observation_pattern = re.compile(r"^P0504196019")
    
    # Regular expression for instrument-specific GTI FITS files
    gti_pattern = re.compile(r"^P\d+_HE_gti\.fits$|^P\d+_ME_gti\.fits$|^P\d+_LE_gti\.fits$")
    
    # Traverse base directory
    for proposal_dir in os.listdir(base_directory):
        proposal_path = os.path.join(base_directory, proposal_dir)
        
        if os.path.isdir(proposal_path) and proposal_pattern.match(proposal_dir):
            for observation_dir in os.listdir(proposal_path):
                observation_path = os.path.join(proposal_path, observation_dir)
                
                if os.path.isdir(observation_path) and observation_pattern.match(observation_dir):
                    pipeline_path = os.path.join(observation_path, "pipeline_output")

                    if os.path.isdir(pipeline_path):
                        for file in os.listdir(pipeline_path):
                            if gti_pattern.match(file):
                                full_path = os.path.join(pipeline_path, file)
                                gti_files.append(full_path)
    
    return gti_files

#Example usage:
gti_files = find_gti_files(".")
print(gti_files)


['./P0504196/P0504196019/pipeline_output/P050419601901_HE_gti.fits', './P0504196/P0504196019/pipeline_output/P050419601901_LE_gti.fits', './P0504196/P0504196019/pipeline_output/P050419601901_ME_gti.fits']


In [47]:
def write_corrected_fits(original_fits, corrected_timestamps=None, corrected_start=None, corrected_stop=None, 
                         is_gti=False, MJDREFI=None, MJDREFF=None, TIMEZERO=None):
    base_name = os.path.basename(original_fits)
    original_dir = os.path.dirname(original_fits)
    
    # Define file names based on type
    if is_gti:
        new_file = os.path.join(original_dir, base_name.replace('.fits', '_orbittimeNO.fits'))
    else:
        new_file = os.path.join(original_dir, base_name.replace('.fits', '_orbittimeNO.fits'))

    try:
        with fits.open(original_fits, mode='readonly') as hdul:
            MJDREF = MJDREFI + MJDREFF

            if is_gti:
                corrected_start = (corrected_start - MJDREF) * 86400 - TIMEZERO
                corrected_stop = (corrected_stop - MJDREF) * 86400 - TIMEZERO
                hdul[1].data['START'] = corrected_start
                hdul[1].data['STOP'] = corrected_stop
            else:
                corrected_met = (corrected_timestamps - MJDREF) * 86400 - TIMEZERO
                hdul[1].data['TIME'] = corrected_met
            
            hdul.writeto(new_file, overwrite=True)
        
        print(f"Corrected FITS file saved as {new_file}")
    
    except Exception as e:
        print(f"Error saving corrected FITS file: {str(e)}")


In [48]:
def process_fits_files(base_directory, asini, porb, ecc, omega_d, t0, t90):
    event_files = find_event_files(base_directory)
    gti_files = find_gti_files(base_directory)
    
    all_input_times = []
    all_corrected_times = []
    
    # Initialize reference values
    MJDREFI = None
    MJDREFF = None
    TIMEZERO = None

    for event_file in event_files:
        with fits.open(event_file) as evt:
            met = evt[1].data['TDB'] 
            if len(met) == 0:
                print(f"Skipping file {event_file}: No time values found.")
                continue
        
            # Extract and store reference values for GTI use
            MJDREFI = evt[1].header['MJDREFI']
            MJDREFF = evt[1].header['MJDREFF']
            TIMEZERO = evt[1].header['TIMEZERO']

            times = ((met + TIMEZERO) / 86400) + MJDREFI + MJDREFF
            all_input_times.append(times)

            corrected_times = Binary_orbit(times, asini, porb, ecc, omega_d, t0=t0, t90=t90)
            all_corrected_times.append(corrected_times)
            print("Romer Delay: ", (times - corrected_times))

            write_corrected_fits(event_file, corrected_timestamps=corrected_times, 
                                 MJDREFI=MJDREFI, MJDREFF=MJDREFF, TIMEZERO=TIMEZERO)

    for gti_file in gti_files:
        if MJDREFI is None or MJDREFF is None or TIMEZERO is None:
            print(f"Skipping GTI file {gti_file}: No event file processed to extract reference values.")
            continue

        with fits.open(gti_file) as gti:
            start_times = ((gti[1].data['START'] + TIMEZERO) / 86400) + MJDREFI + MJDREFF
            stop_times = ((gti[1].data['STOP'] + TIMEZERO) / 86400) + MJDREFI + MJDREFF
        
            corrected_start = Binary_orbit(start_times, asini, porb, ecc, omega_d, t0=t0, t90=t90)
            corrected_stop = Binary_orbit(stop_times, asini, porb, ecc, omega_d, t0=t0, t90=t90)

            write_corrected_fits(gti_file, corrected_start=corrected_start, corrected_stop=corrected_stop, 
                                 is_gti=True, MJDREFI=MJDREFI, MJDREFF=MJDREFF, TIMEZERO=TIMEZERO)

    return all_input_times, all_corrected_times

In [49]:
############
# Input parameters taken from: 
# https://gammaray.nsstc.nasa.gov/gbm/science/pulsars/lightcurves/swiftj0243.html
############

base_directory = "." #Change the data accordingly\
asini = 115.531  # in light-seconds
porb = 27.6943  # in days
ecc = 0.1029
omega_d = -74.05
t0=-1
t90 = 58116.097  # Modified Julian Date

# Run the processing
input_times, corrected_times=process_fits_files(base_directory, asini, porb, ecc, omega_d,t0=t0, t90=t90)

58103.476861347226 Debugging time
starting iterations (max 20)
Mean Anomaly: [-1.22020581 -1.2202058  -1.2202058  ... -1.19775287 -1.19775287
 -1.19775287]
Eccentric Anomaly: [-1.31988363 -1.31988361 -1.31988361 ... -1.29681483 -1.29681483
 -1.29681483]
e:  0.1029 a_sin_i 0.001337164351851852 omega  -1.292416311101801
sinE:  [-0.96868621 -0.96868621 -0.96868621 ... -0.96270127 -0.96270127
 -0.96270127]
cosE:  [0.24828818 0.2482882  0.2482882  ... 0.27056656 0.27056657 0.27056657]
term f:  [-0.00054098 -0.00054098 -0.00054098 ... -0.00056743 -0.00056743
 -0.00056743]
term df: [-0.00026884 -0.00026884 -0.00026884 ... -0.00026578 -0.00026578
 -0.00026578]
Final Correction:  [-0.00054112 -0.00054112 -0.00054112 ... -0.00056758 -0.00056758
 -0.00056758]
iteration 1 completed (max 20)
Mean Anomaly: [-1.22008305 -1.22008303 -1.22008303 ... -1.1976241  -1.1976241
 -1.19762409]
Eccentric Anomaly: [-1.31975764 -1.31975762 -1.31975762 ... -1.29668237 -1.29668237
 -1.29668237]
e:  0.1029 a_sin_i 0

In [ ]:
## Now we need the correction for GTI file. 